# Week 14 - Day 7: Interview Review - RNN/LSTM/GRU

## Learning Objectives
- Master 10 essential RNN/LSTM interview questions
- Understand common mistakes in sequence modeling
- Build a multi-asset LSTM predictor (mini-project)
- Review and consolidate Week 14 concepts

**Date:** Week 14, Day 7  
**Focus:** Interview Preparation & Practical Application

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

plt.style.use('seaborn-v0_8-whitegrid')

---
# Part 1: 10 RNN/LSTM Interview Questions with Answers

Master these questions to ace your quantitative finance interviews.

## Question 1: What is the vanishing gradient problem and how do LSTMs solve it?

### Answer:

**Vanishing Gradient Problem:**
- In standard RNNs, gradients are multiplied through many time steps during backpropagation
- If gradients < 1, they shrink exponentially → vanishing gradients
- If gradients > 1, they explode exponentially → exploding gradients
- Result: Network cannot learn long-term dependencies

**LSTM Solution:**
1. **Cell State (Highway):** Direct path for gradient flow with minimal transformations
2. **Gating Mechanisms:** Control information flow additively, not multiplicatively
3. **Forget Gate:** Decides what to discard from cell state
4. **Input Gate:** Controls new information added to cell state
5. **Output Gate:** Regulates what information is output

The cell state acts as a "conveyor belt" allowing gradients to flow unchanged over many timesteps.

In [ ]:
# Demonstration: Gradient flow comparison
def simulate_gradient_flow(n_timesteps, gradient_factor, is_lstm=False):
    """Simulate gradient magnitude through time steps"""
    gradients = [1.0]  # Initial gradient
    
    for t in range(1, n_timesteps):
        if is_lstm:
            # LSTM: Additive updates preserve gradients better
            # Simplified: forget_gate close to 1 preserves gradient
            forget_gate = 0.9 + np.random.uniform(-0.05, 0.05)
            gradients.append(gradients[-1] * forget_gate)
        else:
            # RNN: Multiplicative updates
            gradients.append(gradients[-1] * gradient_factor)
    
    return gradients

# Compare gradient flow
timesteps = 50
rnn_vanishing = simulate_gradient_flow(timesteps, 0.8, is_lstm=False)
rnn_exploding = simulate_gradient_flow(timesteps, 1.2, is_lstm=False)
lstm_gradients = simulate_gradient_flow(timesteps, 0.8, is_lstm=True)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(rnn_vanishing, 'r-', linewidth=2)
axes[0].set_title('RNN: Vanishing Gradients (factor=0.8)', fontsize=12)
axes[0].set_xlabel('Time Steps')
axes[0].set_ylabel('Gradient Magnitude')
axes[0].set_yscale('log')

axes[1].plot(rnn_exploding, 'orange', linewidth=2)
axes[1].set_title('RNN: Exploding Gradients (factor=1.2)', fontsize=12)
axes[1].set_xlabel('Time Steps')
axes[1].set_ylabel('Gradient Magnitude')
axes[1].set_yscale('log')

axes[2].plot(lstm_gradients, 'g-', linewidth=2)
axes[2].set_title('LSTM: Stable Gradients', fontsize=12)
axes[2].set_xlabel('Time Steps')
axes[2].set_ylabel('Gradient Magnitude')

plt.tight_layout()
plt.show()

print(f"After {timesteps} steps:")
print(f"  RNN Vanishing: {rnn_vanishing[-1]:.2e}")
print(f"  RNN Exploding: {rnn_exploding[-1]:.2e}")
print(f"  LSTM: {lstm_gradients[-1]:.4f}")

## Question 2: Explain the difference between LSTM and GRU. When would you use each?

### Answer:

| Aspect | LSTM | GRU |
|--------|------|-----|
| **Gates** | 3 (Forget, Input, Output) | 2 (Reset, Update) |
| **States** | Cell state + Hidden state | Hidden state only |
| **Parameters** | More (~4x hidden size²) | Fewer (~3x hidden size²) |
| **Training Speed** | Slower | Faster |
| **Memory** | Better for very long sequences | Good for moderate sequences |

**When to use LSTM:**
- Very long sequences (>100 timesteps)
- Complex temporal patterns
- When model capacity is important
- Financial applications with subtle long-term dependencies

**When to use GRU:**
- Shorter sequences
- Limited computational resources
- Smaller datasets (less prone to overfitting)
- Real-time inference requirements

In [ ]:
# Parameter count comparison
input_size = 10
hidden_size = 64

lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
gru = nn.GRU(input_size, hidden_size, batch_first=True)
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

print("Parameter Comparison (input=10, hidden=64):")
print(f"  RNN:  {count_parameters(rnn):,} parameters")
print(f"  GRU:  {count_parameters(gru):,} parameters")
print(f"  LSTM: {count_parameters(lstm):,} parameters")
print(f"\nRatio: LSTM is {count_parameters(lstm)/count_parameters(gru):.2f}x larger than GRU")

## Question 3: What is teacher forcing? What are its advantages and disadvantages?

### Answer:

**Teacher Forcing:**
- Training technique where ground truth is fed as input at each time step
- Instead of using model's own predictions, use actual values

**Advantages:**
1. Faster convergence
2. More stable training
3. Avoids error accumulation during training

**Disadvantages:**
1. **Exposure Bias:** Model never sees its own mistakes during training
2. At inference, model uses its predictions → distribution mismatch
3. Can lead to poor generalization

**Solutions:**
- **Scheduled Sampling:** Gradually decrease teacher forcing probability
- **Professor Forcing:** Use adversarial training to match distributions
- **Curriculum Learning:** Start with teacher forcing, gradually remove

In [ ]:
# Scheduled Sampling Implementation
def scheduled_sampling_prob(epoch, total_epochs, strategy='linear'):
    """Calculate probability of using teacher forcing"""
    if strategy == 'linear':
        return max(0, 1 - epoch / total_epochs)
    elif strategy == 'exponential':
        return 0.99 ** epoch
    elif strategy == 'inverse_sigmoid':
        k = 5  # Steepness
        return k / (k + np.exp(epoch / k))
    return 1.0

epochs = np.arange(100)
linear = [scheduled_sampling_prob(e, 100, 'linear') for e in epochs]
exponential = [scheduled_sampling_prob(e, 100, 'exponential') for e in epochs]
inv_sigmoid = [scheduled_sampling_prob(e, 100, 'inverse_sigmoid') for e in epochs]

plt.figure(figsize=(10, 5))
plt.plot(epochs, linear, label='Linear Decay', linewidth=2)
plt.plot(epochs, exponential, label='Exponential Decay', linewidth=2)
plt.plot(epochs, inv_sigmoid, label='Inverse Sigmoid', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Teacher Forcing Probability')
plt.title('Scheduled Sampling Strategies')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Question 4: How do you handle variable-length sequences in PyTorch?

### Answer:

**Methods:**

1. **Padding + pack_padded_sequence:**
   - Pad sequences to same length
   - Use `pack_padded_sequence` before LSTM
   - Use `pad_packed_sequence` after LSTM
   - Most efficient and commonly used

2. **Masking:**
   - Create attention mask
   - Apply mask in loss calculation

3. **Bucketing:**
   - Group similar-length sequences together
   - Reduces padding waste

In [ ]:
# Variable-length sequence handling demonstration
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

# Create sequences of different lengths
seq1 = torch.randn(5, 3)   # Length 5
seq2 = torch.randn(3, 3)   # Length 3
seq3 = torch.randn(7, 3)   # Length 7
sequences = [seq1, seq2, seq3]
lengths = torch.tensor([5, 3, 7])

# Pad sequences (batch_first=True)
padded = pad_sequence(sequences, batch_first=True, padding_value=0)
print(f"Padded shape: {padded.shape}")  # [3, 7, 3] - batch, max_len, features

# Sort by length (required for pack_padded_sequence)
lengths_sorted, sort_idx = lengths.sort(descending=True)
padded_sorted = padded[sort_idx]

# Pack for efficient LSTM processing
packed = pack_padded_sequence(padded_sorted, lengths_sorted.cpu(), batch_first=True)

# Process with LSTM
lstm = nn.LSTM(input_size=3, hidden_size=16, batch_first=True)
packed_output, (h_n, c_n) = lstm(packed)

# Unpack
output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)
print(f"Output shape: {output.shape}")

# Restore original order
_, unsort_idx = sort_idx.sort()
output_original_order = output[unsort_idx]
print(f"Final output shape: {output_original_order.shape}")

## Question 5: What is the difference between stateful and stateless LSTMs?

### Answer:

| Aspect | Stateless LSTM | Stateful LSTM |
|--------|----------------|---------------|
| **Hidden State** | Reset to zero after each batch | Preserved across batches |
| **Use Case** | Independent sequences | Continuous time series |
| **Training** | Simpler | Requires careful batch ordering |
| **Memory** | Lower | Higher (stores states) |

**Stateful LSTM Use Cases in Finance:**
- Streaming real-time predictions
- Very long sequences that don't fit in memory
- Online learning scenarios

**Implementation Note:**
In PyTorch, pass `(h_0, c_0)` to maintain state; reset manually when needed.

In [ ]:
# Stateful LSTM Example
class StatefulLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.hidden = None
        
    def reset_hidden(self, batch_size):
        """Reset hidden state to zeros"""
        self.hidden = (
            torch.zeros(self.num_layers, batch_size, self.hidden_size),
            torch.zeros(self.num_layers, batch_size, self.hidden_size)
        )
    
    def forward(self, x):
        if self.hidden is None:
            self.reset_hidden(x.size(0))
        
        out, self.hidden = self.lstm(x, self.hidden)
        # Detach to prevent backprop through entire history
        self.hidden = (self.hidden[0].detach(), self.hidden[1].detach())
        return out

# Usage
stateful_lstm = StatefulLSTM(input_size=5, hidden_size=32)

# Process chunks of a long sequence
print("Processing streaming data:")
stateful_lstm.reset_hidden(batch_size=1)

for chunk_idx in range(3):
    chunk = torch.randn(1, 10, 5)  # batch=1, seq_len=10, features=5
    output = stateful_lstm(chunk)
    print(f"  Chunk {chunk_idx + 1}: output shape = {output.shape}, "
          f"hidden mean = {stateful_lstm.hidden[0].mean().item():.4f}")

## Question 6: How do you prevent overfitting in LSTM models?

### Answer:

**Regularization Techniques:**

1. **Dropout:**
   - Regular dropout between layers
   - **NOT** within recurrent connections (disrupts temporal learning)
   - Use `dropout` parameter in `nn.LSTM` (applies between layers)

2. **Recurrent Dropout (Variational):**
   - Same dropout mask across time steps
   - Preserves temporal coherence

3. **Weight Decay (L2 Regularization):**
   - Add to optimizer: `weight_decay=1e-5`

4. **Early Stopping:**
   - Monitor validation loss
   - Stop when no improvement for N epochs

5. **Data Augmentation:**
   - Add noise to inputs
   - Time warping, scaling

6. **Gradient Clipping:**
   - Prevents gradient explosion
   - `torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)`

In [ ]:
# LSTM with proper regularization
class RegularizedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, 
            hidden_size, 
            num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0  # Dropout between layers
        )
        self.dropout = nn.Dropout(dropout)  # Dropout before output
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.dropout(lstm_out[:, -1, :])  # Last timestep + dropout
        return self.fc(out)

# Training with regularization techniques
model = RegularizedLSTM(input_size=5, hidden_size=64, num_layers=2, output_size=1, dropout=0.3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # L2 reg

print("Regularization techniques applied:")
print("  ✓ Dropout between LSTM layers: 0.3")
print("  ✓ Dropout before output layer: 0.3")
print("  ✓ Weight decay (L2): 1e-5")
print(f"\nModel parameters: {count_parameters(model):,}")

## Question 7: How do you choose the sequence length (lookback window) for financial time series?

### Answer:

**Considerations:**

1. **Domain Knowledge:**
   - Trading days in a month (~21 days)
   - Quarterly patterns (~63 days)
   - Annual seasonality (~252 days)

2. **Statistical Methods:**
   - Autocorrelation analysis
   - Partial autocorrelation (PACF)
   - Cross-validation with different windows

3. **Practical Factors:**
   - Computational constraints
   - Memory requirements
   - Longer sequences = more parameters, overfitting risk

4. **Empirical Validation:**
   - Grid search over sequence lengths
   - Monitor both training and validation metrics

**Common Ranges:**
- Intraday: 30-120 observations
- Daily: 20-60 days
- For regime detection: 60-252 days

In [ ]:
# Autocorrelation analysis for sequence length selection
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf

# Get sample data
ticker = yf.Ticker('SPY')
hist = ticker.history(period='2y')
returns = hist['Close'].pct_change().dropna()

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# ACF
plot_acf(returns, ax=axes[0], lags=60, alpha=0.05)
axes[0].set_title('Autocorrelation Function (ACF)')
axes[0].set_xlabel('Lag (Days)')

# PACF
plot_pacf(returns, ax=axes[1], lags=60, alpha=0.05)
axes[1].set_title('Partial Autocorrelation Function (PACF)')
axes[1].set_xlabel('Lag (Days)')

plt.tight_layout()
plt.show()

# Find significant lags
acf_values = acf(returns, nlags=60)
confidence_interval = 1.96 / np.sqrt(len(returns))
significant_lags = np.where(np.abs(acf_values) > confidence_interval)[0]
print(f"Significant ACF lags: {significant_lags[:10]}")
print(f"\nSuggested sequence lengths to try: [20, 30, 40, 60]")

## Question 8: What are attention mechanisms and how do they improve sequence models?

### Answer:

**Attention Mechanism:**
- Allows model to focus on relevant parts of input sequence
- Learns weighted importance of each time step
- Creates direct connections bypassing sequential bottleneck

**Benefits:**
1. **Long-range dependencies:** Direct connections to any timestep
2. **Interpretability:** Attention weights show which inputs matter
3. **Parallel computation:** (for self-attention)
4. **Better gradient flow:** Shorter paths for backprop

**Types:**
- **Bahdanau Attention:** Additive attention
- **Luong Attention:** Multiplicative attention
- **Self-Attention:** Query, Key, Value from same sequence

In [ ]:
# Simple Attention Layer Implementation
class AttentionLayer(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attention = nn.Linear(hidden_size, 1)
        
    def forward(self, lstm_output):
        # lstm_output: [batch, seq_len, hidden_size]
        attention_weights = torch.softmax(
            self.attention(lstm_output).squeeze(-1), dim=1
        )
        # Weighted sum of LSTM outputs
        context = torch.bmm(
            attention_weights.unsqueeze(1), lstm_output
        ).squeeze(1)
        return context, attention_weights

class LSTMWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.attention = AttentionLayer(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        context, attn_weights = self.attention(lstm_out)
        output = self.fc(context)
        return output, attn_weights

# Test
model = LSTMWithAttention(input_size=5, hidden_size=32, output_size=1)
x = torch.randn(2, 20, 5)  # batch=2, seq_len=20, features=5
output, attn = model(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attn.shape}")
print(f"\nAttention weights (sum to 1): {attn[0].sum().item():.4f}")

## Question 9: How do you handle non-stationarity in financial time series for LSTM models?

### Answer:

**The Problem:**
- Financial prices are non-stationary (mean, variance change over time)
- LSTMs assume patterns learned on training data apply to test data

**Solutions:**

1. **Transform to Returns:**
   - `returns = (price_t - price_{t-1}) / price_{t-1}`
   - More stationary than raw prices

2. **Log Returns:**
   - `log_returns = log(price_t / price_{t-1})`
   - Better for longer horizons

3. **Differencing:**
   - First or second order differences

4. **Rolling Normalization:**
   - Z-score with rolling mean and std
   - Adapts to changing market conditions

5. **Volatility Scaling:**
   - Divide by rolling volatility
   - Removes heteroskedasticity

In [ ]:
# Non-stationarity handling techniques
prices = hist['Close'].values

# Different transformations
simple_returns = np.diff(prices) / prices[:-1]
log_returns = np.diff(np.log(prices))

# Rolling z-score normalization
def rolling_zscore(data, window=20):
    rolling_mean = pd.Series(data).rolling(window).mean()
    rolling_std = pd.Series(data).rolling(window).std()
    return ((data - rolling_mean) / rolling_std).values

zscore_returns = rolling_zscore(simple_returns, window=20)

# Volatility scaling
rolling_vol = pd.Series(simple_returns).rolling(20).std().values
vol_scaled = simple_returns / rolling_vol

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

axes[0, 0].plot(prices[-252:])
axes[0, 0].set_title('Raw Prices (Non-Stationary)')

axes[0, 1].plot(simple_returns[-252:])
axes[0, 1].set_title('Simple Returns (More Stationary)')
axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)

axes[1, 0].plot(zscore_returns[-252:])
axes[1, 0].set_title('Rolling Z-Score Normalized')
axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.5)

axes[1, 1].plot(vol_scaled[-252:])
axes[1, 1].set_title('Volatility Scaled Returns')
axes[1, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)

for ax in axes.flat:
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Question 10: How do you evaluate and compare LSTM models for financial prediction?

### Answer:

**Regression Metrics (Price/Return Prediction):**
- MSE, RMSE, MAE
- R² Score
- MAPE (Mean Absolute Percentage Error)

**Directional Metrics:**
- Directional Accuracy (DA)
- Hit Rate

**Financial Metrics:**
- Sharpe Ratio of predictions
- Maximum Drawdown
- Information Coefficient (IC)
- Profit Factor

**Important Considerations:**
1. Use walk-forward validation (not random split)
2. Account for transaction costs
3. Test on out-of-sample period
4. Compare against benchmarks (buy-and-hold, naive forecast)
5. Statistical significance testing (DM test)

In [ ]:
# Comprehensive LSTM evaluation metrics
def evaluate_predictions(y_true, y_pred, returns_true=None):
    """Calculate comprehensive evaluation metrics"""
    results = {}
    
    # Regression metrics
    results['MSE'] = mean_squared_error(y_true, y_pred)
    results['RMSE'] = np.sqrt(results['MSE'])
    results['MAE'] = mean_absolute_error(y_true, y_pred)
    
    # Directional accuracy
    if len(y_true) > 1:
        true_direction = np.sign(np.diff(y_true))
        pred_direction = np.sign(np.diff(y_pred))
        results['Directional_Accuracy'] = np.mean(true_direction == pred_direction)
    
    # Information Coefficient (Spearman correlation)
    from scipy.stats import spearmanr
    ic, _ = spearmanr(y_true, y_pred)
    results['IC'] = ic
    
    # Strategy returns if trading based on predictions
    if returns_true is not None:
        signal = np.sign(y_pred[:-1])  # Position based on prediction
        strategy_returns = signal * returns_true[1:]
        results['Strategy_Return'] = np.sum(strategy_returns)
        results['Sharpe_Ratio'] = np.mean(strategy_returns) / np.std(strategy_returns) * np.sqrt(252)
    
    return results

# Example with synthetic predictions
np.random.seed(42)
y_true = np.cumsum(np.random.randn(100) * 0.01) + 100
y_pred = y_true + np.random.randn(100) * 0.5  # Add noise
returns_true = np.diff(y_true) / y_true[:-1]

metrics = evaluate_predictions(y_true, y_pred, returns_true)

print("Model Evaluation Metrics:")
print("-" * 40)
for metric, value in metrics.items():
    print(f"  {metric}: {value:.4f}")

---
# Part 2: Common Mistakes in Sequence Modeling

Avoid these pitfalls to build robust LSTM models.

## Mistake 1: Data Leakage in Time Series

**Problem:** Using future information in training

**Common Sources:**
- Normalizing with entire dataset statistics
- Random train/test split instead of temporal split
- Feature engineering using future data

In [ ]:
# ❌ WRONG: Normalize with all data
def wrong_normalize(data):
    scaler = MinMaxScaler()
    return scaler.fit_transform(data.reshape(-1, 1))  # Uses future data!

# ✅ CORRECT: Normalize with only past data
def correct_normalize(train_data, test_data):
    scaler = MinMaxScaler()
    scaler.fit(train_data.reshape(-1, 1))  # Fit only on training
    train_scaled = scaler.transform(train_data.reshape(-1, 1))
    test_scaled = scaler.transform(test_data.reshape(-1, 1))  # Transform test
    return train_scaled, test_scaled, scaler

# ❌ WRONG: Random split
def wrong_split(X, y):
    from sklearn.model_selection import train_test_split
    return train_test_split(X, y, test_size=0.2, random_state=42)  # Shuffles!

# ✅ CORRECT: Temporal split
def correct_split(X, y, train_ratio=0.8):
    split_idx = int(len(X) * train_ratio)
    return X[:split_idx], X[split_idx:], y[:split_idx], y[split_idx:]

print("Data Leakage Prevention:")
print("  ✓ Use temporal train/test split")
print("  ✓ Fit scalers on training data only")
print("  ✓ Calculate features using only past data")
print("  ✓ Use walk-forward validation")

## Mistake 2: Improper Sequence Creation

**Problem:** Creating sequences that leak information or misalign targets

In [ ]:
# ❌ WRONG: Target overlaps with input
def wrong_create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length - 1])  # WRONG: Last input = target!
    return np.array(X), np.array(y)

# ✅ CORRECT: Target is next value after sequence
def correct_create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])  # CORRECT: Predict NEXT value
    return np.array(X), np.array(y)

# Demonstration
sample_data = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
seq_len = 3

X_wrong, y_wrong = wrong_create_sequences(sample_data, seq_len)
X_correct, y_correct = correct_create_sequences(sample_data, seq_len)

print("Sample Data:", sample_data)
print(f"\n❌ WRONG sequence creation (target overlaps):")
print(f"   X[0]: {X_wrong[0]} → y[0]: {y_wrong[0]} (3 is in both!)")
print(f"\n✅ CORRECT sequence creation:")
print(f"   X[0]: {X_correct[0]} → y[0]: {y_correct[0]} (predicting next value)")

## Mistake 3: Not Using Gradient Clipping

**Problem:** Gradient explosion leads to NaN values and unstable training

In [ ]:
# ❌ WRONG: No gradient clipping
def wrong_training_step(model, optimizer, criterion, X, y):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()  # Gradients can explode!
    return loss

# ✅ CORRECT: With gradient clipping
def correct_training_step(model, optimizer, criterion, X, y, max_norm=1.0):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    
    # Clip gradients
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
    
    optimizer.step()
    return loss

# Monitor gradient norms
def get_gradient_norm(model):
    total_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            total_norm += p.grad.data.norm(2).item() ** 2
    return total_norm ** 0.5

print("Gradient Clipping Best Practices:")
print("  ✓ Use clip_grad_norm_ with max_norm=1.0 to 5.0")
print("  ✓ Monitor gradient norms during training")
print("  ✓ If gradients consistently hit max, reduce learning rate")

## Mistake 4: Wrong Hidden State Initialization

**Problem:** Hidden states not properly reset between batches or initialized with wrong shapes

In [ ]:
# ❌ WRONG: Incorrect hidden state shape
def wrong_init_hidden(batch_size, hidden_size, num_layers):
    # Missing num_layers dimension!
    h0 = torch.zeros(batch_size, hidden_size)
    c0 = torch.zeros(batch_size, hidden_size)
    return (h0, c0)

# ✅ CORRECT: Proper initialization
def correct_init_hidden(batch_size, hidden_size, num_layers, device='cpu'):
    # Shape: (num_layers * num_directions, batch, hidden_size)
    h0 = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    c0 = torch.zeros(num_layers, batch_size, hidden_size).to(device)
    return (h0, c0)

# Test
batch_size, hidden_size, num_layers = 32, 64, 2
h0, c0 = correct_init_hidden(batch_size, hidden_size, num_layers)
print(f"Correct hidden state shape: h0={h0.shape}, c0={c0.shape}")
print(f"\nExpected: (num_layers={num_layers}, batch={batch_size}, hidden={hidden_size})")

## Mistake 5: Ignoring Bidirectional Output Shape

**Problem:** Not accounting for doubled hidden size in bidirectional LSTMs

In [ ]:
# Bidirectional LSTM output shapes
input_size = 5
hidden_size = 32
batch_size = 16
seq_len = 20

# Unidirectional
lstm_uni = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=False)

# Bidirectional
lstm_bi = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)

x = torch.randn(batch_size, seq_len, input_size)

out_uni, (h_uni, c_uni) = lstm_uni(x)
out_bi, (h_bi, c_bi) = lstm_bi(x)

print("Output Shapes:")
print(f"  Unidirectional output: {out_uni.shape}")
print(f"  Bidirectional output:  {out_bi.shape}  ← 2x hidden size!")
print(f"\nHidden State Shapes:")
print(f"  Unidirectional h: {h_uni.shape}")
print(f"  Bidirectional h:  {h_bi.shape}  ← 2x num_layers!")

# ✅ CORRECT: Account for bidirectional in FC layer
fc_correct = nn.Linear(hidden_size * 2, 1)  # 2x for bidirectional
print(f"\n✅ Correct FC input size for bidirectional: {hidden_size * 2}")

## Mistake 6: Not Handling NaN Values Properly

**Problem:** NaN values in financial data cause training to fail

In [ ]:
# Proper NaN handling for financial data
def prepare_financial_data(df):
    """Robust data preparation pipeline"""
    df = df.copy()
    
    # 1. Check for NaN
    nan_count = df.isna().sum().sum()
    print(f"NaN values found: {nan_count}")
    
    # 2. Forward fill (most appropriate for time series)
    df = df.ffill()
    
    # 3. Backward fill remaining (at start)
    df = df.bfill()
    
    # 4. Check for infinite values
    inf_count = np.isinf(df.select_dtypes(include=[np.number])).sum().sum()
    print(f"Infinite values found: {inf_count}")
    
    # 5. Replace infinite with NaN then fill
    df = df.replace([np.inf, -np.inf], np.nan).ffill().bfill()
    
    # 6. Final check
    assert df.isna().sum().sum() == 0, "Data still contains NaN!"
    
    return df

# Test
test_df = pd.DataFrame({
    'price': [100, np.nan, 102, 103, np.inf, 105],
    'volume': [1000, 1100, np.nan, 1300, 1400, 1500]
})

print("Before cleaning:")
print(test_df)
print("\nAfter cleaning:")
cleaned_df = prepare_financial_data(test_df)
print(cleaned_df)

---
# Part 3: Mini-Project - Multi-Asset LSTM Predictor

Build an LSTM model that predicts returns for multiple assets simultaneously.

In [ ]:
# Download multi-asset data
tickers = ['SPY', 'QQQ', 'IWM', 'TLT', 'GLD']  # Diversified assets
start_date = '2018-01-01'
end_date = '2023-12-31'

print("Downloading multi-asset data...")
data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Close']
data = data.dropna()

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
print(f"Assets: {list(data.columns)}")

# Display price evolution
fig, ax = plt.subplots(figsize=(14, 6))
normalized = data / data.iloc[0] * 100
for col in normalized.columns:
    ax.plot(normalized.index, normalized[col], label=col, linewidth=1.5)
ax.set_title('Normalized Asset Prices (Base = 100)')
ax.set_xlabel('Date')
ax.set_ylabel('Normalized Price')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Prepare data for multi-asset prediction
class MultiAssetDataset:
    def __init__(self, prices_df, seq_length=30, train_ratio=0.8):
        self.seq_length = seq_length
        self.n_assets = len(prices_df.columns)
        self.asset_names = list(prices_df.columns)
        
        # Calculate returns
        self.returns = prices_df.pct_change().dropna()
        
        # Temporal split
        split_idx = int(len(self.returns) * train_ratio)
        self.train_returns = self.returns.iloc[:split_idx]
        self.test_returns = self.returns.iloc[split_idx:]
        
        # Normalize using training data only
        self.scaler = MinMaxScaler(feature_range=(-1, 1))
        self.scaler.fit(self.train_returns.values)
        
        train_scaled = self.scaler.transform(self.train_returns.values)
        test_scaled = self.scaler.transform(self.test_returns.values)
        
        # Create sequences
        self.X_train, self.y_train = self._create_sequences(train_scaled)
        self.X_test, self.y_test = self._create_sequences(test_scaled)
        
        # Convert to tensors
        self.X_train = torch.FloatTensor(self.X_train)
        self.y_train = torch.FloatTensor(self.y_train)
        self.X_test = torch.FloatTensor(self.X_test)
        self.y_test = torch.FloatTensor(self.y_test)
        
    def _create_sequences(self, data):
        X, y = [], []
        for i in range(len(data) - self.seq_length):
            X.append(data[i:i + self.seq_length])
            y.append(data[i + self.seq_length])  # Predict all assets
        return np.array(X), np.array(y)
    
    def get_dataloaders(self, batch_size=32):
        train_dataset = TensorDataset(self.X_train, self.y_train)
        test_dataset = TensorDataset(self.X_test, self.y_test)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        return train_loader, test_loader

# Create dataset
dataset = MultiAssetDataset(data, seq_length=30, train_ratio=0.8)

print(f"Training samples: {len(dataset.X_train)}")
print(f"Test samples: {len(dataset.X_test)}")
print(f"Input shape: {dataset.X_train.shape}")
print(f"Output shape: {dataset.y_train.shape}")

In [ ]:
# Multi-Asset LSTM Model
class MultiAssetLSTM(nn.Module):
    def __init__(self, n_assets, hidden_size=64, num_layers=2, dropout=0.2):
        super().__init__()
        self.n_assets = n_assets
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Shared LSTM layers
        self.lstm = nn.LSTM(
            input_size=n_assets,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Attention layer
        self.attention = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.Tanh(),
            nn.Linear(hidden_size // 2, 1)
        )
        
        # Output layer - predict all assets
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, n_assets)
        )
        
    def forward(self, x):
        # x: [batch, seq_len, n_assets]
        lstm_out, _ = self.lstm(x)  # [batch, seq_len, hidden]
        
        # Attention
        attn_weights = torch.softmax(self.attention(lstm_out).squeeze(-1), dim=1)
        context = torch.bmm(attn_weights.unsqueeze(1), lstm_out).squeeze(1)
        
        # Output
        output = self.fc(context)
        return output, attn_weights

# Initialize model
model = MultiAssetLSTM(
    n_assets=dataset.n_assets,
    hidden_size=64,
    num_layers=2,
    dropout=0.2
).to(device)

print(f"Model Architecture:")
print(model)
print(f"\nTotal parameters: {count_parameters(model):,}")

In [ ]:
# Training function with proper techniques
def train_model(model, train_loader, test_loader, epochs=50, lr=0.001, patience=10):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    train_losses = []
    test_losses = []
    best_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            output, _ = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # Validation
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                output, _ = model(X_batch)
                test_loss += criterion(output, y_batch).item()
        
        test_loss /= len(test_loader)
        test_losses.append(test_loss)
        
        scheduler.step(test_loss)
        
        # Early stopping
        if test_loss < best_loss:
            best_loss = test_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.6f}, "
                  f"Test Loss: {test_loss:.6f}, LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break
    
    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return train_losses, test_losses

# Train the model
train_loader, test_loader = dataset.get_dataloaders(batch_size=32)

print("Training Multi-Asset LSTM...\n")
train_losses, test_losses = train_model(
    model, train_loader, test_loader, 
    epochs=100, lr=0.001, patience=15
)

In [ ]:
# Plot training history
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(train_losses, label='Training Loss', linewidth=2)
ax.plot(test_losses, label='Validation Loss', linewidth=2)
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE Loss')
ax.set_title('Multi-Asset LSTM Training History')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Evaluate model
model.eval()
all_predictions = []
all_targets = []
all_attention = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        output, attn = model(X_batch)
        all_predictions.append(output.cpu().numpy())
        all_targets.append(y_batch.numpy())
        all_attention.append(attn.cpu().numpy())

predictions = np.vstack(all_predictions)
targets = np.vstack(all_targets)
attention_weights = np.vstack(all_attention)

# Inverse transform
predictions_original = dataset.scaler.inverse_transform(predictions)
targets_original = dataset.scaler.inverse_transform(targets)

# Calculate metrics per asset
print("Per-Asset Performance Metrics:")
print("=" * 60)

results = {}
for i, asset in enumerate(dataset.asset_names):
    pred = predictions_original[:, i]
    true = targets_original[:, i]
    
    mse = mean_squared_error(true, pred)
    mae = mean_absolute_error(true, pred)
    
    # Directional accuracy
    true_dir = np.sign(true)
    pred_dir = np.sign(pred)
    da = np.mean(true_dir == pred_dir)
    
    # Information coefficient
    from scipy.stats import spearmanr
    ic, _ = spearmanr(true, pred)
    
    results[asset] = {'MSE': mse, 'MAE': mae, 'DA': da, 'IC': ic}
    print(f"{asset:6} | MSE: {mse:.6f} | MAE: {mae:.6f} | DA: {da:.2%} | IC: {ic:.4f}")

In [ ]:
# Visualize predictions vs actual
fig, axes = plt.subplots(len(dataset.asset_names), 1, figsize=(14, 3*len(dataset.asset_names)))

for i, (ax, asset) in enumerate(zip(axes, dataset.asset_names)):
    ax.plot(targets_original[:100, i] * 100, label='Actual', alpha=0.8, linewidth=1.5)
    ax.plot(predictions_original[:100, i] * 100, label='Predicted', alpha=0.8, linewidth=1.5)
    ax.set_title(f'{asset} - Returns Prediction (First 100 samples)')
    ax.set_ylabel('Return (%)')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

axes[-1].set_xlabel('Time Step')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize attention patterns
avg_attention = attention_weights.mean(axis=0)

plt.figure(figsize=(12, 4))
plt.bar(range(len(avg_attention)), avg_attention, color='steelblue', alpha=0.7)
plt.xlabel('Time Step (Days Back)')
plt.ylabel('Average Attention Weight')
plt.title('LSTM Attention Pattern - Which Days Matter Most?')
plt.xticks(range(0, len(avg_attention), 5))
plt.grid(True, alpha=0.3, axis='y')

# Highlight most important days
top_days = np.argsort(avg_attention)[-5:]
for day in top_days:
    plt.bar(day, avg_attention[day], color='coral', alpha=0.9)

plt.tight_layout()
plt.show()

print(f"\nMost important lookback days: {sorted(top_days)}")
print(f"Most recent day attention: {avg_attention[-1]:.4f}")

In [ ]:
# Simple trading strategy backtest
def backtest_strategy(predictions, actual_returns, transaction_cost=0.001):
    """Backtest long-only strategy based on predictions"""
    n_assets = predictions.shape[1]
    
    # Strategy: Go long assets with positive predicted return
    signals = (predictions > 0).astype(float)
    
    # Equal weight among selected assets
    weights = signals / (signals.sum(axis=1, keepdims=True) + 1e-8)
    
    # Portfolio returns
    portfolio_returns = (weights * actual_returns).sum(axis=1)
    
    # Transaction costs (when positions change)
    turnover = np.abs(np.diff(weights, axis=0)).sum(axis=1)
    costs = np.concatenate([[0], turnover * transaction_cost])
    
    net_returns = portfolio_returns - costs
    cumulative_returns = (1 + net_returns).cumprod()
    
    # Metrics
    total_return = cumulative_returns[-1] - 1
    sharpe = np.mean(net_returns) / np.std(net_returns) * np.sqrt(252)
    max_dd = (cumulative_returns / cumulative_returns.cummax() - 1).min()
    
    return {
        'cumulative_returns': cumulative_returns,
        'total_return': total_return,
        'sharpe_ratio': sharpe,
        'max_drawdown': max_dd,
        'daily_returns': net_returns
    }

# Run backtest
strategy_results = backtest_strategy(predictions_original, targets_original)

# Benchmark: Equal weight buy and hold
equal_weight_returns = targets_original.mean(axis=1)
benchmark_cum = (1 + equal_weight_returns).cumprod()

print("Strategy Backtest Results:")
print("=" * 40)
print(f"Total Return:    {strategy_results['total_return']:.2%}")
print(f"Sharpe Ratio:    {strategy_results['sharpe_ratio']:.2f}")
print(f"Max Drawdown:    {strategy_results['max_drawdown']:.2%}")
print(f"\nBenchmark (Equal Weight B&H):")
print(f"Total Return:    {benchmark_cum[-1] - 1:.2%}")

In [ ]:
# Plot strategy performance
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Cumulative returns
axes[0].plot(strategy_results['cumulative_returns'], label='LSTM Strategy', linewidth=2)
axes[0].plot(benchmark_cum, label='Equal Weight Benchmark', linewidth=2, alpha=0.7)
axes[0].set_title('Strategy vs Benchmark Performance')
axes[0].set_ylabel('Cumulative Return')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Drawdown
drawdown = strategy_results['cumulative_returns'] / strategy_results['cumulative_returns'].cummax() - 1
axes[1].fill_between(range(len(drawdown)), drawdown * 100, 0, alpha=0.5, color='red')
axes[1].set_title('Strategy Drawdown')
axes[1].set_xlabel('Trading Days')
axes[1].set_ylabel('Drawdown (%)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# Part 4: Week 14 Summary

## Key Concepts Covered This Week

### Day 1: RNN Fundamentals
- Recurrent architecture and hidden states
- Backpropagation through time (BPTT)
- Vanishing/exploding gradient problem

### Day 2: LSTM Networks
- Cell state and gating mechanisms
- Forget, Input, and Output gates
- Long-term dependency learning

### Day 3: GRU Architecture
- Simplified gating (Reset, Update)
- Comparison with LSTM
- When to use GRU vs LSTM

### Day 4: Sequence-to-Sequence Models
- Encoder-Decoder architecture
- Multi-step forecasting
- Attention mechanisms

### Day 5: Advanced Techniques
- Bidirectional LSTMs
- Stacked LSTM layers
- Regularization strategies

### Day 6: Practical Applications
- Price prediction
- Volatility forecasting
- Feature engineering for sequences

### Day 7: Interview Review (Today)
- 10 essential interview questions
- Common mistakes to avoid
- Multi-asset LSTM project

## Quick Reference: LSTM Interview Cheat Sheet

```
LSTM GATES:
├── Forget Gate: f_t = σ(W_f · [h_{t-1}, x_t] + b_f)
├── Input Gate: i_t = σ(W_i · [h_{t-1}, x_t] + b_i)
├── Cell Candidate: c̃_t = tanh(W_c · [h_{t-1}, x_t] + b_c)
├── Cell Update: c_t = f_t * c_{t-1} + i_t * c̃_t
├── Output Gate: o_t = σ(W_o · [h_{t-1}, x_t] + b_o)
└── Hidden State: h_t = o_t * tanh(c_t)

GRU GATES:
├── Reset Gate: r_t = σ(W_r · [h_{t-1}, x_t] + b_r)
├── Update Gate: z_t = σ(W_z · [h_{t-1}, x_t] + b_z)
├── Candidate: h̃_t = tanh(W_h · [r_t * h_{t-1}, x_t] + b_h)
└── Output: h_t = (1 - z_t) * h_{t-1} + z_t * h̃_t

PARAMETER COUNTS:
├── LSTM: 4 * (input_size + hidden_size + 1) * hidden_size
├── GRU:  3 * (input_size + hidden_size + 1) * hidden_size
└── RNN:  (input_size + hidden_size + 1) * hidden_size

KEY HYPERPARAMETERS:
├── Sequence Length: 20-60 for daily data
├── Hidden Size: 32-256
├── Num Layers: 1-3
├── Dropout: 0.1-0.5
├── Learning Rate: 1e-4 to 1e-2
└── Gradient Clip: 1.0-5.0
```

## Week 14 Checklist

### Core Concepts ✅
- [ ] Understand RNN architecture and BPTT
- [ ] Explain vanishing gradient problem
- [ ] Describe LSTM gates in detail
- [ ] Compare LSTM vs GRU
- [ ] Implement attention mechanism

### Practical Skills ✅
- [ ] Build LSTM model in PyTorch
- [ ] Handle variable-length sequences
- [ ] Apply regularization techniques
- [ ] Use gradient clipping
- [ ] Implement early stopping

### Financial Applications ✅
- [ ] Preprocess time series correctly
- [ ] Avoid data leakage
- [ ] Choose appropriate sequence length
- [ ] Evaluate with financial metrics
- [ ] Backtest trading strategies

In [ ]:
# Final summary statistics
print("=" * 60)
print("WEEK 14 - DAY 7 COMPLETE: Interview Review")
print("=" * 60)
print("\n📚 Covered:")
print("   • 10 Essential RNN/LSTM Interview Questions")
print("   • 6 Common Mistakes in Sequence Modeling")
print("   • Multi-Asset LSTM Predictor Project")
print("\n🎯 Mini-Project Results:")
print(f"   • Assets: {', '.join(dataset.asset_names)}")
print(f"   • Strategy Sharpe: {strategy_results['sharpe_ratio']:.2f}")
print(f"   • Max Drawdown: {strategy_results['max_drawdown']:.1%}")
print("\n🚀 Next Steps:")
print("   • Week 15: Attention & Transformers for Finance")
print("   • Practice implementing from scratch")
print("   • Review interview questions daily")
print("=" * 60)